In [2]:
import pandas as pd
import numpy as np
import copy as cp
import darc_core
import pickle
import sys
import hashlib as hs
import base64 as b64
import os
import random as rd
import datetime as dt
from darc_core.metrics import Metrics
from darc_core.utils import M_COL, T_COL, T_COL_IT, NB_GUESS, SIZE_POOL
from darc_core.preprocessing import round1_preprocessing
from darc_core.utils import check_format_trans_file

ModuleNotFoundError: No module named 'pandas'

In [3]:
%%time
df = pd.read_csv("./ground_truth.csv", parse_dates=["date"])
df["hours"]= pd.to_datetime(df["hours"], format = '%H:%M%S').dt.hour
df["month"]=df["date"].dt.month
df["year"]=df["date"].dt.year
df["id_user"]=df["id_user"].astype(str)
df["price"]=df["price"].astype("float")
df["qty"]=df["qty"].astype("float")
df_dict = df.T.to_dict('list')
df.head()

CPU times: user 26.8 s, sys: 744 ms, total: 27.6 s
Wall time: 30.1 s


In [3]:
df.head()

,id_user,date,hours,id_item,price,qty,month,year
0,17850,2010-12-01,8,85123A,2.55,6.0,12,2010
1,17850,2010-12-01,8,71053,3.39,6.0,12,2010
2,17850,2010-12-01,8,84406B,2.75,8.0,12,2010
3,17850,2010-12-01,8,84029G,3.39,6.0,12,2010
4,17850,2010-12-01,8,84029E,3.39,6.0,12,2010


In [1]:
df

NameError: name 'df' is not defined

In [214]:
%%time
def mergeIdentical(df,df_dict):
    """ Merge "same" transaction into one line. Assign "DEL" to duplicates
        Transactions are "identical" if their keys are equal
        
        :map: key:[id_user,date,id_item] value:[qty_cumulative,count]
    """
    map={}
    for d in range(0,len(df)):
        if(map.get((df["id_user"][d],df["date"][d],df["id_item"][d]))):
            map[(df["id_user"][d],df["date"][d],df["id_item"][d])][0] += df_dict[d][5]
            map[(df["id_user"][d],df["date"][d],df["id_item"][d])][1] += 1
            df_dict[d][0]="DEL"
            df_dict[d][1]=df_dict[d][1].strftime("%Y-%m-01")
            df_dict[d][2]=""
            df_dict[d][3]=""
            df_dict[d][4]=""
            df_dict[d][5]=""
        else:
            map[(df["id_user"][d],df["date"][d],df["id_item"][d])] = [df_dict[d][5],1]
    
    dfn = pd.DataFrame.from_dict(df_dict,orient="index")
    dfn = dfn.rename(columns={0:"id_user",1:"date",2:"hours",3:"id_item",4:"price",5:"qty",6:"month",7:"year"})    
  
    return dfn ,map

dfn, map= mergeIdentical(df,df_dict)
save_dfn_merge=dfn.copy()

CPU times: user 37.3 s, sys: 205 ms, total: 37.5 s
Wall time: 37.8 s


In [218]:
dfn[dfn["id_user"]=="DEL"]

,id_user,date,hours,id_item,price,qty,month,year
47,DEL,2010-12-01,,,,,12,2010
48,DEL,2010-12-01,,,,,12,2010
49,DEL,2010-12-01,,,,,12,2010
50,DEL,2010-12-01,,,,,12,2010
51,DEL,2010-12-01,,,,,12,2010
61,DEL,2010-12-01,,,,,12,2010
62,DEL,2010-12-01,,,,,12,2010
63,DEL,2010-12-01,,,,,12,2010
64,DEL,2010-12-01,,,,,12,2010
66,DEL,2010-12-01,,,,,12,2010


In [80]:
def split(df, partition, column): 
    """
        Split df[partition] over column on median 
        :return: List[[IndexRange]]*2 {left ,right}
    """
    dfp = df["id_user"][partition]
    sanitizedPartition = dfp.index[dfp != "DEL"]
    dfp = df[column][sanitizedPartition]
    if (column in categorical):
        values = dfp.unique()
        dfp.sort_values(inplace=True)
        lv=set(values[:len(values)//2])
        rv=set(values[len(values)//2:])
        return dfp.index[dfp.isin(lv)],dfp.index[dfp.isin(rv)]
    else:
        median=dfp.median()
        dfp.sort_values(inplace=True)
        dfl=dfp.index[dfp<median]
        dfr=dfp.index[dfp>=median]
        return(dfl,dfr)

In [81]:
def is_k_anonymous (df, partition, k=1):
    if ((len(partition)<k)):
        return False
    return True

In [82]:
%%time
dfn["id_item"]=dfn["id_item"].astype("category")
categorical = set ({'id_item'})
def partition_dataset(df, is_valid):
    """
        Greedy search
        Split dataframe over {year,month,id_item} 
        :return: List[[IndexRange]] 
    """
    finished_partitions=[]
    partitions=[df.index]
    while partitions:    
        partition = partitions.pop(0)
        columns = ["year","month", "id_item"]
        for column in columns:
            lp, rp = split(df,partition,column)
            if not is_valid(df,lp) or not is_valid(df,rp):
                continue
            partitions.extend((lp,rp))
            break
        else:
            finished_partitions.append(partition)
    return finished_partitions
finished_partitions = partition_dataset(dfn, is_k_anonymous)
save_finished_partitions=finished_partitions.copy()


CPU times: user 14min 31s, sys: 13.7 s, total: 14min 45s
Wall time: 15min 31s


In [103]:
with open('finished_partitions', 'wb') as fp:
    pickle.dump(finished_partitions, fp)

In [4]:
with open ('finished_partitions', 'rb') as fp:
    finished_partitions = pickle.load(fp)

In [222]:
%%time
def make_l_diverse(df_dict,partitions,l=9):
    filtered=[]
    for p in finished_partitions:
        array = []
        for i in p:
            if(df_dict[i][0] != "DEL" and df_dict[i][0] not in array):
                array.append(df_dict[i][0])
        if len(array)<l:
            for i in p:
                df_dict[i][0]="DEL"
                df_dict[i][1]=df_dict[i][1].strftime("%Y-%m-01")
                df_dict[i][2]=""
                df_dict[i][3]=""
                df_dict[i][4]=""
                df_dict[i][5]=""
        else:
            filtered.append(p)
            
    dfn = pd.DataFrame.from_dict(df_dict,orient="index")
    dfn = dfn.rename(columns={0:"id_user",1:"date",2:"hours",3:"id_item",4:"price",5:"qty",6:"month",7:"year"})  
    
    return dfn,filtered

dfn, finished_partitions = make_l_diverse(df_dict,finished_partitions)
save_diversity=finished_partitions.copy()

CPU times: user 1.8 s, sys: 40.9 ms, total: 1.84 s
Wall time: 1.86 s


In [124]:
def verify_dict_diverse():
    i=0
    for r in recup_diversed: 
        array = (r == finished_partitions[i])
        for a in array:
            if (a==False):
                return False
        i+=1
    return True
print(verify_dict_diverse())

True


In [223]:
dfn[dfn["id_user"]=="DEL"]

,id_user,date,hours,id_item,price,qty,month,year
13,DEL,2010-12-01,,,,,12,2010
14,DEL,2010-12-01,,,,,12,2010
19,DEL,2010-12-01,,,,,12,2010
22,DEL,2010-12-01,,,,,12,2010
23,DEL,2010-12-01,,,,,12,2010
29,DEL,2010-12-01,,,,,12,2010
41,DEL,2010-12-01,,,,,12,2010
42,DEL,2010-12-01,,,,,12,2010
47,DEL,2010-12-01,,,,,12,2010
48,DEL,2010-12-01,,,,,12,2010


In [106]:
%%time
dfn["id_item"]=dfn["id_item"].astype("category")
categorical = set ({'id_item'})
finished_partitions=save_finished_partitions
def make_l_diverse(df,partitions,l=9):
    """
         DEL partitions with l-diversity < l
    """
    i=0
    filtered=[]
    while(len(partitions)):
        if (df.loc[partitions[0],"id_user"].nunique()<l):
            df.loc[partitions[0],"id_user"]="DEL"
            if(isinstance(df.loc[partitions[0],["date"]].squeeze(),dt.date)):
                df.loc[partitions[0],["date"]]=df.loc[partitions[0],["date"]].squeeze().strftime('%Y-%m-01')
            else:
                df.loc[partitions[0],["date"]]=df.loc[partitions[0],["date"]].squeeze().apply(lambda x: x.strftime('%Y-%m-01'))
            df.loc[partitions[0],["hours","id_item"]]="" 
            partitions.pop(0)           
        else:
            filtered.append(partitions.pop(0))       
    return dfn, filtered
dfn, finished_partitions = make_l_diverse(dfn,finished_partitions)
save_diversity=finished_partitions.copy()

CPU times: user 31min 22s, sys: 38.9 s, total: 32min 1s
Wall time: 34min 12s


In [108]:
with open('diverse_partitions', 'wb') as fp:
    pickle.dump(finished_partitions, fp)

In [117]:
with open ('diverse_partitions', 'rb') as fp:
    recup_diversed = pickle.load(fp)

In [225]:
def shuffleDates(df,partition):
   
    date_array = []
    date_shuffled = []
    for i in partition:
        date_array.append(df_dict[i][1])
    while(len(date_array)):
        i=rd.randrange(len(date_array))
        date_shuffled.append(date_array[i])
        date_array.pop(i)
    k=0
    for i in partition:
        df_dict[i][1]=date_shuffled[k]
        k+=1
    
    return 1

In [130]:
def shuffleDates(df,partition):
    """
         For same partition, swap {dates,qty} randomly.
    """
    date_array=df.loc[partition,"date"].copy().squeeze()
    date_array.reset_index(drop=True,inplace=True) 
    date_shuffled=[]
    while(len(date_array)):
        i=rd.randrange(len(date_array))
        date_shuffled.append(date_array[i])
        date_array.drop(i,inplace=True)
        date_array.reset_index(drop=True,inplace=True)
    df.loc[partition,"date"]=date_shuffled
    return df

In [9]:
%%time
def build_anonymized_dataset(df,partitions): 
    for p in partitions:
        mean_hour=0
        mean_qty=0
        for i in p:
            mean_hour+=df_dict[i][2]
            mean_qty+=df_dict[i][5]
            #shuffleDates(df,p)
        for i in p:
            df_dict[i][2]=mean_hour/len(p)
            df_dict[i][5]=mean_qty/len(p)
    dfn = pd.DataFrame.from_dict(df_dict,orient="index")
    dfn = dfn.rename(columns={0:"id_user",1:"date",2:"hours",3:"id_item",4:"price",5:"qty",6:"month",7:"year"})  
    
    return dfn
dfn=build_anonymized_dataset(df,finished_partitions)
save_builded=dfn.copy()

CPU times: user 2 s, sys: 51.5 ms, total: 2.05 s
Wall time: 2.12 s


In [6]:
%%time
def build_anonymized_dataset(df,partitions): 
    for p in partitions:
        mean_hour=0
        for i in p:
            mean_hour+=df_dict[i][2]
            #shuffleDates(df,p)
        for i in p:
            df_dict[i][2]=mean_hour/len(p)
            
    dfn = pd.DataFrame.from_dict(df_dict,orient="index")
    dfn = dfn.rename(columns={0:"id_user",1:"date",2:"hours",3:"id_item",4:"price",5:"qty",6:"month",7:"year"})  
    
    return dfn
dfn=build_anonymized_dataset(df,finished_partitions)

CPU times: user 1.53 s, sys: 53.7 ms, total: 1.59 s
Wall time: 1.62 s


In [10]:
dfn

,id_user,date,hours,id_item,price,qty,month,year
0,17850,2010-12-01,12.686667,85123A,2.55,17.600000,12,2010
1,17850,2010-12-01,12.142857,71053,3.39,6.642857,12,2010
2,17850,2010-12-01,12.764706,84406B,2.75,7.823529,12,2010
3,17850,2010-12-01,12.800000,84029G,3.39,8.033333,12,2010
4,17850,2010-12-01,12.675676,84029E,3.39,8.472973,12,2010
5,17850,2010-12-01,12.878788,22752,7.65,10.151515,12,2010
6,17850,2010-12-01,12.318182,21730,4.25,4.863636,12,2010
7,17850,2010-12-01,12.384615,22633,1.85,9.500000,12,2010
8,17850,2010-12-01,12.000000,22632,1.85,12.130435,12,2010
9,13047,2010-12-01,12.646341,84879,1.69,23.609756,12,2010


In [14]:
%%time
def build_anonymized_dataset(df,partitions):
    """
        Generalization of Columns {dates,qty,hours}
    """
    for p in partitions:
        df.loc[p,["hours"]]=df.loc[p,["hours"]].mean().values[0]
        df=shuffleDates(df,p)
    return df
dfn=build_anonymized_dataset(dfn,finished_partitions)
save_dfn=dfn.copy()

CPU times: user 24min 45s, sys: 37.6 s, total: 25min 23s
Wall time: 26min 1s


In [11]:
%%time
dfn.id_user=dfn.id_user.astype(str)
dfn_copy=dfn.copy()

def generateSalts():
    """ 
        Generate Salt{256b} Table : Mij = Salt for user i in month j 
    """
    # Set of {1..12} (number of the month)
    ordered_months = list(range(1,14))
    # Set of all unique user's IDs
    ids = dfn_copy["id_user"].unique()
    ids = ids[ids != "DEL"]
    # Generate salt table
    salt_table=pd.DataFrame(columns=ordered_months,index=ids)
    salt_table.set_index(ids,inplace=True)
    for i in ids:
        for j in ordered_months:
            # Generates unique b64 values
            salt_table[j][i]=os.urandom(256)
    return salt_table

salt_table = generateSalts()
salt_dict = salt_table.T.to_dict('list')

CPU times: user 9.42 s, sys: 747 ms, total: 10.2 s
Wall time: 11 s


In [251]:
salt_dict["17850"][11]

b"\xe4\xf6\xc2\xd0\x18<.;\x8a\xb3_\x1d\x0c\xfcq\xe0\x99\t7\x92\xb6\xd9\xfa\x18\xa7\xa8\xc9\xad\x03\x90\xc3\x12>\xa4\xc8\xc5\xea\xd3\x983\x95\x18\xc6\xb3\x0ex`\xea\x832\xa8~E\xe6\\\xf7\xffb\x838/\xc1b\x8c\x1c\\\x11\xc1\x0f\x96\xf9\x03\xf2\xcd\x89!)\xb0\xd0\xc79\xfcvZ26\xf1\xed\xcb\x99\x98d\x8e\xa8\x197/\xdb\xe7\x8c\xb2>\x0f\x95\xa7\xabP2u\xc1\xea9ee-\x06\x0fc\xb5E\xbc\x85k0\x8e\r\xe22\xb2\xd4\xd8\x1f\t\x07\xe8\x06z\x93,\xb3s\xe5\xecq\xf2\x856\x13.\xaa\xb7\xf4\x06\xc6'\xbb\xf0\xfa\xe1\x1e\x91P\x8d\x98V#9\xec\xd3\xcf\xd4:7\xe7\x86\x0c\xe9\xf2\x8e\x10s\xcc)2}N\xb2\xd4\xa9\xd6\xc7\xe1\x9bBed,b\xb7\xc3\xa6S\xd4\xe5\x91s\x02\x1d\xae7\xf2\x04H#y[\xee@\x84\x94@&\x13g\xe0|\xc40\xb5\xa7\xd2m\x11\x06O\xc3F{\xd0aM\xddU\xce\xe3*\xda(|\xbexL\xc4c\xfe\xbb"

In [12]:
def generateF(df):
    users=df["id_user"].unique().tolist()
    months=list(range(1,14))
    F = pd.DataFrame(index=users,columns=months)
    for col in F.columns:
        F[col].values[:] = "DEL"
    F_dict = F.T.to_dict('list')
    
    return F,F_dict

F,F_dict = generateF(df)

In [268]:
F.to_csv("F_Real.csv", index=True)

In [15]:
%%time
def populateF():
    """
        Apply hashme for all rows
    """
    dfn_copy.sort_values(["id_user","month"])
    salt_table.sort_index(inplace=True)

    dfn["id_user"]=dfn_copy.apply(lambda x: hashme(x["id_user"],x["month"],x["year"]), axis=1)
populateF()

Progress 99 -- Count : 306555 / 307054

In [176]:
salt_table.loc[:,2:13]

,1,2,3,4,5,6,7,8,9,10,11,12
17850,b'+\x10\xc2V\xd1\xec2\'\xf4\xd4\r\x91\xc7\xcf\...,"b""\xcd\xa4P$=\x0e\x1c\xcf\xcc4\x1c\xee}p\xb3\x...","b""\xa6\x7f\x82i\xbf\xf0\xfd\x8a\x19|!U\xa2\x0f...",b'\xf0\xad$\x89\xef\x05ldz\n\x05\x8e\x96\xdd\x...,"b""\xd8\x9f\xbe\xff\xa0\x08}{\xf5\x9e\xf9\xce\x...",b'\xd0F\x06>\xee.I\x08\xc5\x8aJ\xd3=J\xa6Q\x0f...,b'8\t6\xd7\xf5~\xe8\xc6\xb3\xfe\xc3\x00\xa4\xe...,"b""\x04F\xe42\x1e\xf3\xcc\xf5Bpe\xd7~'\x19/\xf6...","b'\x8a\t@\n\xd4\xaf\xfe\xbf\x81*\xb0;""\xa0p\x1...","b'w^\x87\x85\xe5)\x9b1\xb3T,\xadn\x17\xb8\xb30...",b')\xc9\xbfo>\xf8\xcfy\x94\x1b\xc2RG\x92J\xd2X...,b'\xa8\xec\x83\xae(Z\xaaba;\xf2\xdcb|P\xa7\xb2...
13047,b'\x96\x85\x02\x1d\xb28k\'P\xbe\xc4h\x028\xdc\...,b'\x9f\x1er\xf0.\xb6\x95\xe7\xb8_|\x86\xda\xfc...,b'\xf8\x9aB\xd3\x1e\xfd&s\xf5\x15_\xb8G\x8a2k(...,"b""?L\xe7Cn\xbaY\xf7\xb6PG9\xa1\xddE\xcb\xac\xa...",b'h\xe3\xdc\xf1\x10\x0e8\xfd\x1euP\xa5\x0f\xe3...,b'\x98\xd5y\x98\x93\xce\xab34\xcd0\xc2\x8b\xe6...,"b""w\x8b\x119\x19\x06\x0fG\xb3\xb9\x9f*\x11\xbd...","b'7\x99\x9a\xe4+\x94\xa6\x80\xff\x90#\x172""\x0...","b'R\xaa5\x81\xb8Rw\x90\x02\x7f\xa6\x8b\x04"".H\...","b""\\\xc7\xb9F\xc1UnV\x0c\xa9\xd2\x0b\x18ZIE!D\...","b""\xec\xdf\xc7\xcaz\xc0\xe0\xe8\x80>\xac\xe7\x...",b'\xb7\xc33t%\xdb\xa6r\x16\xe2\xfb\x98i\x88*\x...
12583,"b""p\xb5\x9cA\xc0{,\xc9\x87r\x89-O\x8c0\x06rz;\...",b'\xb8R\xbfQ+\xbeH:\xa1.M\x16`z\x17@\xd7+\xbb6...,b'\x0b\xbe\tb\x8e\xf4s \xa2\xa4\xd0\x1f\xc4|iH...,b'\xa2\x1dw\x02\xef\xf4\xcf\xc2Pk\xe1l_\xe0R\x...,"b""\x1e\xec\x01\xb9\x99V\xf58\xa2\x15L\xb5\x95|...",b'~G7\x14\xd9!\xfa\x9b\xf0\x85\xfa\xfd\x1d\xab...,"b""\x83j\x8a;\x99\x15\x83x[\xf1(\xfaj\x90\xe1P\...","b""\xdd\x86\x97\x1d\x02\x90Fh\x80p\xe2\x87\xda5...",b'\x0c\x12\xc2\xc5\xc9FT\x1b\xe3b\x1b%\xf5\x13...,b'\xa9\xb1\x18\xb1(\x83\x19\xa4\xc6\x04\xb5\x8...,b'[[$\x0c\xd3\x9b\xdb\xa3\xba\xfcr\xe6\xbag\x9...,"b""\x1eO\xbf\xa0\x9a$O\x15=\x1f8\x15\x82\xff\xd..."
13748,b'\x8d\xaa\xa0>y\x91\x9a\xb6h\x97t\xa2o\xf0\x1...,"b""\xd3RQB(\xc6yz+d[\xd2\x1fs\xb1\x1b\x9f&\xe1v...",b'|\'\x05\xed\x80\xf7q\x1c\xf2k\x97\xca\xd1F8A...,b':\xcb\x92G0U)\x92\xa8\xc1<\xb6b\xd3\xf2/n\x0...,b'\xbeL\x7f\x00\xd5=E\xab+M\xd4\xff\xec\x8e\xb...,"b'\x1bh\xa5""\x19\x16>n\xaf\xa9\x07\x89\xe8H\x0...","b""\xa7\x12f\x1e\x9durx\xc8P~\xd4\x17\xdeFx\xe4...",b'\xe9\x98\x88\x1e\x94\xb2\x14\xd6\xc8\xa9\xfb...,b'\xfc\\\x02t\x9c\x9a=\xc1{\xc3\xeb\xbb\x10{a\...,b'@\x1a\xc9\xca\xa7\x19t?\x1f\x04\xf0\x97R\x11...,"b""|\xf3vW\xae\x9d\x8b\xce7\x19\xdc\\\x98\xca\x...",b'@\xd2(p\x18\xdf\x9ej\xde\xb9c\x15;\xab\x13\x...
15100,"b'\x91{K\x19\x00\x03""\xa4\xd5\x9b\xeeU\x05<\xf...",b'\xc7\x93\xe0 \xbb\xedR\x87s\x88\xde\x15\xd4F...,b'\x87\x03}\x04\x17\xe7Mq\xa9\x97X\x07\xaa\xc9...,"b'\xd0\x90D=D\xa8\xea\xcb"".\xf2\xffnmC\x01\xd3...",b'\x0fy\xec\x9e\xa5\xf9\xee\xcb\x0e\xea\x11=\x...,"b""a\xe8^\xb1]\x98\xf4\xa4=\x0f\x99\xc8\x07|\xb...","b'*6T->\xe1\x86\xb0\x1b<\x03,\x02n\xcbbH\x97t^...",b'*\xb3\t6F%\xb9?\xc1\xc5\xef\xee[\xd8z\xfc\xd...,b'\xcbf#\xc6\x8f\x1e\xd8\x10\xeaz\xf5q\xf2re\x...,"b'\x07""\xba\xdc\x12\x95^t\x0f`\xd0\t\xac\x94Fj...",b'CX\x9bpOH\xad\xff\xa8.\xc1\xe9\xbaj\x08\xea\...,b'\xf8r\x08\xfd\x9d\xc6\x9f\xaf\x19\x0b\x8cV\x...
15291,b'\xd0\x03S\xa5\xbc\x0e\x8f5mm\xbb\xf3\x19V\xc...,b'\xcb\x83\xd4\x1d\x17\x12\xd3M3\xa5\x95o\x85\...,b'\xaf\xd6\xb7\xdaH-\xfes\xaa\x86\xeed[\xe2\xe...,b'c\xb5\xcf\x90\xaf\xb3\xb4\x84wg\x82\xdb}\x82...,b'\x81p\xed\xb5\xe8\xa8\x90\xb9e}\xcc\x92l\'\x...,b'\xcbu\x93v\x1b\xc5D\xf4\x16o\xe8c]\xc0\xa3$\...,"b""\n\xe2\xaf\x1f\r\xfd+xa\xb37\x17\x91<:\xe5\x...",b':\xd8a\xa2\xd0\xedj\xf0\x1a_\xe5$&-\n\x10\x1...,b'xj\x9d\xe2\xa7\xd7\xf2\x85}\x19m\xb8\x96\x7f...,"b""\x9d\xbf\xef<\xb3)\x04\x9c\xc5\xc1\xd1\x8a\x...",b'L\x19\xbd11G\x0e\xe0\x1ee\xf2\x16\x9eK\x9cJ\...,b'\xcc\x9f\nf\xc0v\x13\xb1\xb7y\xaa\x84\xc0\xc...
14688,b'[\x99\xaf~G\xf2C\xd4\xe1A\x82`*\x1e\xcdMS\xa...,"b""\xf9^E\xff\x07\xb1\x05\xd0\xde@\x10%\xac\x9f...","b""}\xbc\xf61\x89\xfdn\xad\x9f\xda\x96\xeft*\x8...",b'\x15\x15Z\'\xc5{\xbe\xa5g\x94e\xb7\xd0}\xf7\...,b'\xb3\x07\xa8p\xa6\xa5\x91cO\xb1Qp\xb6\x0

In [261]:
months=list(range(1,14))
month_dict={ i : months[i] for i in range(0, len(months) ) }
F = pd.DataFrame.from_dict(F_dict,orient="index")
F = F.rename(columns=month_dict)  
F

,1,2,3,4,5,6,7,8,9,10,11,12,13
17850,b707fbff42a17f9503f6e59e6c6f10953742325704645a...,DEL,DEL,DEL,DEL,DEL,DEL,DEL,DEL,DEL,DEL,DEL,DEL
13047,f763c320ffb13f7d8de5a20fc71baf2da17617a8220cd8...,DEL,d6eb683af61a075c52e617d11c8311e47d00a337978d3a...,d9dc64f0072e06dbe2b0e3327d057ad9eea73e770066b1...,DEL,bbd1be10c10d4daa8c583197df16a51488ae61f7744362...,f64da5164902791dcbfa0c0a55f83a16b314919d994fd3...,DEL,1e58cc63b9869bf376a52f8982d9afa593d4438ab00c6f...,DEL,9a93225d9f198ed15982ecc0f8bab7624d1467e19344bc...,9d27e55c5dce0f0fa42f5641ead72309304d77b29913c6...,DEL
12583,e581ea29bf72e645a4eba92fd45ce876c2626967c95210...,e04630c03b03e20b7f707010a39d6e4ae68377a17a109a...,73e4cb6a68b315f51912cf3b8f255623eef5c66a6f9753...,3b26fee8740368e3542945230e7c130756ed6c75dc0844...,DEL,8081ae2bb424fbcbfbbecef6810590c3502f191ab294e1...,8236e758025608d258927ea293a8ee66ac1af2c2e48733...,82b3b805d906ade9eded79ddfcdd98b314c2f06abbbae9...,ae502404635d14c40e13fd5871f73bd67d68ec5b8b53f6...,27a1ac205d3ceff16c50a2ae442254967262cde8d65997...,59bbbb68ed2ca8ae0d398a2d71d2433b6286067e95993c...,1ce24bb6a6dddb89a81a9d3e55ea5843a233e26deb2efc...,088e1ca9e18318ee5be052b95e53327aaa49b29df2e1a8...
13748,44f5e675aa15bd895cb899da3a9907b8a7cd191d253297...,DEL,DEL,DEL,8e14ef6a2255666247f4b1506a14b2a7567291bdcf2640...,DEL,DEL,DEL,DEL,9d2dce0ac11d53b6f131e9edb733a8747eb3fa1ba784a8...,DEL,DEL,DEL
15100,c1b5aced20a9108f91c554f900f40f0eaba0b81f7bd85f...,DEL,DEL,DEL,DEL,DEL,DEL,DEL,DEL,DEL,DEL,DEL,DEL
15291,86f3d3a47079e3adb55a5cd5938514af697dd6e1584cb7...,3f01329e85637b20be11e1d3f9726cfb3f7f7fa36b4727...,DEL,79c89f3a5e843e1b2f6220f3a282875005e0814a92a360...,46d779c99e91919f308d1446d73044ac4f4b91cbca6a2e...,DEL,f064a0c7abaea16fef10e613b0caa0312d7ada008cc964...,DEL,DEL,b1ca3d2b455a3e11342d2a272b1877188328c9bd23102f...,DEL,83090c9a1f2610b60649d3c7cf7eeb41e8be0c4a72aa32...,DEL
14688,f44517f7b75006a3fc0acd3aadc3daa7da2bc47012d21b...,46e5ffe5292f3e253c3f0afebfc2e73324ca8b01a1cf03...,934d0a2493813e9375ea9e4e22072821f8e66fe742d01d...,ae445137e93bd22b18f5fb608f02b2ce2d63a95704b98e...,a30102da9b629e30a333c4ecbc9aabffba27b7148f9044...,ae4576d1e45cc786fc72452b7d469a8f585c1faf24f7bc...,d9d83cb28c4e532fb5826bfd50d27c55a1adf0ec4de53f...,0f6a856839adf913eeed9be42fb108cbccaa0337bfc17c...,70435024f0e470d369dd0b09633fa0557034e6f36ab35d...,9f5a6350747cda37503e07516d7a2d6cfb9e5cd915ad94...,8fa0999ecf65d49d2752abd5817668abf801ff0cc7a176...,DEL,f4ec542ef0307dd365b4d50237af502c6e97f43ca94c24...
17809,f66db69d98977e3746e4963fe990a33fb09fe878ead5e8...,DEL,d1e3ccb78d9452a39a8c6dbf200d49f841520fd0e52d5e...,DEL,de0c64ae19471a4d06e3fac777386bd8ebafd4048c8978...,3db80d7e74915692602a54747b8a0cc4f6eca7c5ea75f1...,ae5f01e9d35a097e1bf4bf938abedad9d66401e4458726...,DEL,a9758541c6d820bc8b5d31e9980192956aa13ab7b18690...,DEL,38f655babd598609931497a2ecc92c94e535b1471ac5ea...,8efd4b4afc3f0229d9d21300b8737d965629451f83f397...,DEL
16098,ef5e63707765aabf7777eb71f63b57dee3396ff332e882...,DEL,7639b9ee61670ff247cd43f2443b3ed60598cea972c9ed...,DEL,8a04e6e39ab3de63deb096d8adfc0c77f05be85e27730b...,DEL,d17b4e4412fc681d89926d0cb398832516e62369d51a3b...,9d8fcb8791ea9f3f42b6cbbc40026b412e2360fbf0eb9b...,923f00d2c77a0424f61788d1ce9be4e8d3243e04c6d7e9...,0fe9cd8048bb1b97ceefb548c946f9b4f747bc626befd5...,DEL,DEL,DEL
18074,c882cac36e73c93dfb11bfae6e1993ea36cc754585e857...,DEL,DEL,DEL,DEL,DEL,DEL,DEL,DEL,DEL,DEL,DEL,DEL


In [14]:
def hashme(id_user, month, year): 
    """
        :return: Hash[user,month] = SHA256(Salt[user,month] + id_user)
    """
    if(id_user=="DEL"):
        return id_user
    else:
        hashme.counter+=1
        percent=(hashme.counter/307054)*100 
        sys.stdout.write("\rProgress %i -- Count : %i / 307054" % (percent,hashme.counter))
        sys.stdout.flush()
        if(year==2010 and month==12 ):
            r = hs.sha256(salt_dict[id_user][0] + id_user.encode()).hexdigest()
            F_dict[id_user][0]=r
        else:
            r = hs.sha256(salt_dict[id_user][month] + id_user.encode()).hexdigest()
            F_dict[id_user][month]=r
        return r
#Static Counter for ProgressBar
hashme.counter=0

In [206]:
%%time
def doHash():
    """
        Apply hashme for all rows
    """
    dfn_copy.sort_values(["id_user","month"])
    salt_table.sort_index(inplace=True)
    dfn["id_user"]=dfn_copy.apply(lambda x: hashme(x["id_user"],x["month"],x["year"]), axis=1)
doHash()
hash_dfn=dfn.copy()

Progress 0 -- Count : 673 / 307054

KeyboardInterrupt: 

In [262]:
hash_dfn=dfn.copy()

In [263]:
%%time
def hour_to_int(hours):
    if(isinstance(hours,float)):
        return int(hours)
def parse_dates(date):
    if(isinstance(date,dt.date) and not pd.isnull(date)):
        return date.strftime('%Y/%m/%d')
dfn["date"]=dfn["date"].astype("datetime64[ns]")
dfn["qty"]=df["qty"].astype("int")
dfn["price"]=df["price"].round(2)
dfn["date"]=dfn["date"].apply(lambda x: x.strftime('%Y/%m/%d'))
dfn["hours"]=dfn.apply(lambda x: hour_to_int(x["hours"]), axis=1)
dfn.drop(["month","year"],axis=1,inplace=True)
dfn.sort_values(["date"],inplace=True)
dfn.to_csv("./atxf.csv",index=False)
dfa=pd.read_csv("./atxf.csv")
dfa

CPU times: user 10.8 s, sys: 255 ms, total: 11.1 s
Wall time: 11.2 s


In [264]:
%%time
ground_truth, submission = round1_preprocessing(
    "./ground_truth.csv", "./atxf.csv"
)
check_format_trans_file(ground_truth, submission)

CPU times: user 13.5 s, sys: 308 ms, total: 13.8 s
Wall time: 13.8 s


In [266]:
%%time
metric = Metrics(ground_truth, submission)
scores = metric.scores()
scores

CPU times: user 24min 16s, sys: 16.6 s, total: 24min 33s
Wall time: 29min 15s


In [267]:
scores

[0.8612072109602361,
 0,
 0.8049433761446219,
 0.0704618101,
 0.429715352,
 0.2097,
 0.001236,
 0.000285,
 0.000285,
 0.000951,
 0.000285,
 0.000951,
 0.002631]

In [77]:
scores = metric.scores_util()
scores

[0.8568029966868055,
 0,
 0.7850147308995272,
 0.0,
 0.431553882,
 0.2057,
 0.028296,
 0.000177,
 8.8e-05,
 0.014767,
 8.8e-05,
 0.01291,
 0.00336]

In [4]:
dfn=pd.read_csv("./atx1.csv", parse_dates=["date"])

In [9]:
dfn[dfn["id_user"]=="DEL"]["qty"].value_counts

<bound method IndexOpsMixin.value_counts of 1         24
2         24
4         24
15         4
19         2
27        12
29        48
31        48
33        12
43        12
44        12
50        12
51         1
53         1
57         1
58         1
59         2
60         2
63        12
64         6
67         2
70         1
76        25
77        25
83        25
85        24
92         8
94         1
96         1
97         1
          ..
306763    25
306812    12
306813    12
306815     6
306829     2
306836     4
306846    12
306861    12
306864    12
306867    24
306920    12
306924    24
306947    48
306949    12
306950     4
306951    12
306953    12
306955    12
306957    96
306971     8
306972    12
306973    24
306974    12
306976    12
306993    15
306994    15
307026    20
307035    12
307043    12
307049    12
Name: qty, Length: 63166, dtype: int64>

In [10]:
mask=dfn["id_user"]=="DEL"
dfn.loc[mask,"date"]=""

In [22]:
dfn.loc[mask,["qty","price","hours","id_item"]]=""

In [80]:
def parse_dates(date):
    if(isinstance(date,dt.date)):
        return date.strftime('%Y/%m/%d')
dfn["date"]=dfn.apply(lambda x: parse_dates(x["date"]), axis=1)

In [40]:
dfn.to_csv("./atx2.csv",index=False)

In [87]:
dfn=hash_dfn.copy()

In [29]:
dfn["date"]=dfn["date"].astype("datetime64[ns]")

In [95]:
dfn[dfn["id_user"]=="DEL"]["date"].value_counts()

2011-11-01    3017
2011-10-01    2455
2010-12-01    2154
2011-09-01    1532
2011-05-01    1466
2011-03-01    1441
2011-06-01    1437
2011-08-01    1412
2011-01-01    1384
2011-04-01    1359
2011-02-01    1357
2011-07-01    1352
2011-12-01     792
Name: date, dtype: int64

In [22]:
"""
File: metrics.py
Author: Antoine Laurent
Email: laurent.antoine@courrier.uqam.ca
Github: https://github.com/Drayer34
Description: class for all re-identification metrics
"""

import sys
from collections import OrderedDict
import time
import math
from multiprocessing import Pool
from pathos.multiprocessing import ProcessingPool as PPool
from functools import partial

import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from scipy.sparse import coo_matrix

class Metrics(object):

    """Super class Metrics for ReidentificationMetrics and UtilityMetrics. It genreate the S data
    from AT one.

    Attributes are :
        :_users: M table containing all users present in the transaction data T (pandas DataFrame).
        :_ground_truth: T table containing all transaction of all user for one year (pandas DataFrame).
        :_anonymized: S table, the anonymized version of the _ground_truth (pandas DataFrame).
        :_users_t_col: the name of the columns in the csv file M.
        :_gt_t_col: the name of the columns in the csv file T.
        :_current_score: current score calculated by the metric already processed.
    """

    def __init__(self, T, AT, M_col=M_COL, T_col=T_COL, T_col_it=T_COL_IT):
        """
        :_users: M table containing all users present in the transaction data T (pandas DataFrame).
        :_ground_truth: T table containing all transaction of all user for one year (pandas DataFrame).
        :_anonymized: S table, the anonymized version of the _ground_truth (pandas DataFrame).
        :_users_t_col: the name of the columns in the csv file M.
        :_gt_t_col: the name of the columns in the csv file T.
        :_gt_t_col: the name of the columns in the csv file T for iteration.
        :_current_score: current score calculated by the metric already processed.
        """
        self._ground_truth = T
        self._users = pd.DataFrame(self._ground_truth.id_user.drop_duplicates().sort_values(), columns=["id_user"]).reset_index(drop=True)
        self._anon_trans = AT
        self._users_t_col = M_col
        self._gt_t_col = T_col
        self._gt_t_col_it = T_col_it
        self._current_score = []

        #  TODO: Cast all data into a fonction  <07-12-18, yourname> #
        self._ground_truth[self._gt_t_col['id_user']] = self._ground_truth[self._gt_t_col['id_user']].apply(str)
        self._anon_trans[self._gt_t_col['id_user']] = self._anon_trans[self._gt_t_col['id_user']].apply(str)

        self._anonymized = self.generate_S_data()

        self._f_orig = self.f_orig()


    def scores_util(self):
        return [self._e1_metric(), self._e2_metric(), self._e3_metric(), self._e4_metric(), self._e5_metric(), self._e6_metric()]

    def scores_reid(self):
        return [self._s1_metric(), self._s2_metric(), self._s3_metric(), self._s4_metric(), self._s5_metric(), self._s6_metric(), self._s7_metric()]

    def scores(self):
        return self.scores_util()

    def generate_S_data(self):
        """Generate S data from AT data.
        :returns: S

        """
        data = self._anon_trans.copy()
        # Remove NaN value from DataFrame
        data = data.dropna()
        # Remove 'DEL' row in DataFrame
        data = data[data[self._gt_t_col['id_user']] != "DEL"]
        #  TODO:check si il y a une seed pour l'aléa  <30-05-18, yourname> #
        data = data.sample(frac=1).reset_index(drop=True)

        return data

    def month_passed(self, date):
        """ Get the month from a date, month should be between 0 and 11

        :date: a date in format YYYY/MM/DD
        :return: integer between 0 and 11 """
        return 0 if date.split('/')[0] == '2010' else int(date.split('/')[1])

    def f_orig(self):
        """Generate the F file for the original data, to compare it with the F^ file.

        :returns: F file original

        """

        # Initialization
        f_orig = pd.DataFrame(columns=['id_user', 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12])
        f_orig.id_user = self._ground_truth[self._gt_t_col['id_user']].value_counts().index
        f_orig = f_orig.sort_values('id_user').reset_index(drop=True)
        f_orig.loc[:, [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]] = "DEL"


        seen = set()
        for row in self._ground_truth.itertuples():
            id_orig = row[self._gt_t_col_it['id_user']]
            month = self.month_passed(row[self._gt_t_col_it['date']])
            id_ano = self._anon_trans.loc[row[0], self._gt_t_col['id_user']]
            item = "{}-{}-{}".format(id_orig, month, id_ano)
            if item not in seen and id_ano != "DEL":
                seen.add(item)
                f_orig.loc[f_orig.id_user == id_orig, month] = id_ano

        return f_orig

    def compare_f_files(self, f_hat):
        """Compare the two F files to compute the difference and thus the score

        :f: the original f file to compare (pandas DataFrame)
        :f_hat: the guessed f file computed by the metric or adversary (pandas DataFrame)

        :returns: score
        """

        #tps1 = time.clock()
        map_error = 0
        score = 0
        count = 0
        total = 0

        #we want the same list of users
        if set(self._f_orig['id_user']).difference(set(f_hat['id_user'])):
            map_error = 1

        if map_error == 0:
            for row in self._f_orig.itertuples():
                # Compare each tuple, if they are egual over all month then gain 12 points
                # One points per similarities
                f_ori_tuple = row[1:]
                f_hat_tuple = tuple(f_hat[f_hat['id_user'] == row[1]].iloc[0])
                for i in range(1, 13):
                    if f_ori_tuple[i] != "DEL":
                        total += 1
                        count += self._compare_row_f_file(f_ori_tuple[i], f_hat_tuple[i])
        #            if f_ori_tuple[i] == f_hat_tuple[i] and f_ori_tuple[i] != "DEL":
        #                count += 1


        if map_error == 0:
            score += round(float(count)/float(total), 6)

        return score

    def _compare_row_f_file(self, row_orig, row_sub):
        """
        Check if row_orig is a substring of row_sub, i.e did the participant submit
        the good id among all id submitted
        """
        row_sub = row_sub.split(':')
        count = 0
        for i in range(len(row_sub)):
            if row_orig in row_sub[i]:
                count = (NB_GUESS - i)/NB_GUESS
        return count

    def _gen_value_id_dic(self, attrs):
        """Generate the dictionaty which associate the value of the attributes attrs in the
        DataFrame to an user ID.

        :attrs: the list of attibutes to check for creating the value
        :return: dictionary of value:id
        """
        value_dic = {}
        for row in self._anonymized.itertuples():
            # Create the value with all the row[attrs] concat with ":"
            value = ':'.join([str(row[elt]) for elt in attrs])
            value_dic[value] = row[self._gt_t_col_it['id_user']]
        return value_dic

    def _guess_inialisation(self):
        """Generate a virgin F^ file with DEL on each column for each id

        :return: the dictionary of id:pseudos
        """
        guess = OrderedDict()
        for row in self._users.itertuples():
            # Fill dic[id] with DEL
            guess[str(row[self._users_t_col['id_user']])] = ['DEL' for i in range(13)]
        return guess

    def _tronc_product_id(self, num):
        """ Tronc the product ID to the number num.
            Example :
                id_prod = ABCDEF and num = 2
                result = AB

        :num: the number of characters to keep.
        """
        col_id_item = self._gt_t_col['id_item']
        self._anonymized.loc[:, col_id_item] = self._anonymized.loc[:, col_id_item]\
                                            .apply(lambda s: s[:min(len(s), num)])
        self._ground_truth.loc[:, col_id_item] = self._ground_truth.loc[:, col_id_item]\
                                            .apply(lambda s: s[:min(len(s), num)])

    def _evaluate(self, attrs):
        """ Evaluate the similtude between T and S on attributs attrs.

        :attrs: attributes to check.
        :return: F^ the guess of Pseudo for each user and each month.
        """

        dic_value_anon = self._gen_value_id_dic(attrs)
        guess = self._guess_inialisation()
        for row in self._ground_truth.itertuples():
            #create the concat of the attributes to watch
            value = ':'.join([str(row[i]) for i in attrs])
            id_user = row[self._gt_t_col_it['id_user']]
            if value in dic_value_anon.keys():
                #recover month of the transaction
                month = self.month_passed(row[self._gt_t_col_it['date']])
                #affect pseudo for id_user where value == value
                guess[id_user][month] = dic_value_anon[value]

        f_hat = pd.DataFrame(guess).transpose()
        f_hat = f_hat.reset_index()
        f_hat = f_hat.rename(columns={'index':'id_user'})

        return f_hat

    def _subset(self, month):
        """
        TODO
        """
        subset1 = time.clock()
        # On définit le masque en fonction du mois, beacoup de cas -> le code est dégeu
        if month != 0 :
            if month < 9:
                start = '2011-0'+str(month)+'-01'
                end = '2011-0'+str(month+1) +'-01'
            if month == 9:
                start = '2011-0'+str(month)+'-01'
                end = '2011-10-01'
            if month == 12:
                start = '2011-'+str(month)+'-01'
                end = '2012-01-01'
            else:
                start = '2011-'+str(month)+'-01'
                end = '2011-'+str(month+1) +'-01'
        else:
            start = '2010-12-01'
            end = '2010-12-31'
        self._ground_truth['date'] = pd.to_datetime(self._ground_truth['date'])
        mask1 = (self._ground_truth['date'] >= start) & (self._ground_truth['date'] < end)
        self._anonymized['date'] = pd.to_datetime(self._anonymized['date'])
        mask2 = (self._anonymized['date'] >= start) & (self._anonymized['date'] < end)

        t_month=self._ground_truth.loc[mask1]
        a_month=self._anonymized.loc[mask2]

        subset2 = time.clock()
        return t_month, a_month, month

    @staticmethod
    def _compute_score(a, t):
        """
        TODO : Find a way to put it in a lambda
        """
        return len(a.intersection(t))/len(t)

    def _find_k_guess(self, t_month, a_month, month):
        """
        TODO
        """
        #k_guess1 = time.clock()

        #Fabrication du vecteur d'item pour chaque utilisateur
        t_month_group=t_month.groupby('id_user')['id_item'].apply(set)
        a_month_group=a_month.groupby('id_user')['id_item'].apply(set)

        # Début du calculs des coefficients de similiarité.
        d_month=dict()
        d_month['month'] = month

        for id_t in t_month_group.index:
            list_guess = list()
            guess = str()
            inter_a_t = a_month_group.apply( lambda x : self._compute_score(x, t_month_group.loc[id_t]))
            top_guess = inter_a_t.nlargest(NB_GUESS)
            for index in top_guess.index :
                 guess += str(index) + ':'
            d_month[id_t] = guess[:-1]
        #k_guess2 = time.clock()
        return d_month

    def _s1_metric(self):
        """Calculate metric S1, comparing date and quantity buy on each row.
        Update the current score value

        :returns: the score on this metric (between 0 and 1)

        """
        date_col = self._gt_t_col_it['date']
        qty_col = self._gt_t_col_it['qty']

        f_hat = self._evaluate([date_col, qty_col])

        score = self.compare_f_files(f_hat)
        # Add the score to the global score for this metric
        self._current_score.append(score)

        return score

    def _s2_metric(self):
        """Calculate metric S2, comparing date and quantity buy on each row.

        :returns: the score on this metric (between 0 and 1)

        """
        id_item_col = self._gt_t_col_it['id_item']
        price_col = self._gt_t_col_it['price']

        f_hat = self._evaluate([id_item_col, price_col])

        score = self.compare_f_files(f_hat)
        # Add the score to the global score for this metric
        self._current_score.append(score)

        return score

    def _s3_metric(self):
        """Calculate metric S3, comparing date and quantity buy on each row.

        :returns: the score on this metric (between 0 and 1)

        """
        id_item_col = self._gt_t_col_it['id_item']
        qty_col = self._gt_t_col_it['qty']

        f_hat = self._evaluate([id_item_col, qty_col])

        score = self.compare_f_files(f_hat)
        # Add the score to the global score for this metric
        self._current_score.append(score)

        return score

    def _s4_metric(self):
        """Calculate metric S4, comparing date and quantity buy on each row.

        :returns: the score on this metric (between 0 and 1)

        """
        date_col = self._gt_t_col_it['date']
        id_item_col = self._gt_t_col_it['id_item']

        f_hat = self._evaluate([date_col, id_item_col])

        score = self.compare_f_files(f_hat)
        # Add the score to the global score for this metric
        self._current_score.append(score)

        return score

    def _s5_metric(self):
        """Calculate metric S5, comparing date and quantity buy on each row.

        :returns: the score on this metric (between 0 and 1)

        """
        id_item_col = self._gt_t_col_it['id_item']
        price_col = self._gt_t_col_it['price']
        qty_col = self._gt_t_col_it['qty']

        f_hat = self._evaluate([id_item_col, price_col, qty_col])

        score = self.compare_f_files(f_hat)
        # Add the score to the global score for this metric
        self._current_score.append(score)

        return score

    def _s6_metric(self):
        """Calculate metric S6, comparing date and quantity buy on each row.

        :returns: the score on this metric (between 0 and 1)

        """
        id_item_col = self._gt_t_col_it['id_item']
        date_col = self._gt_t_col_it['date']
        price_col = self._gt_t_col_it['price']

        f_hat = self._evaluate([id_item_col, date_col, price_col])

        score = self.compare_f_files(f_hat)
        # Add the score to the global score for this metric
        self._current_score.append(score)

        return score

    def _s7_metric(self):
        """
        TODO
        """
        def _reid_multi(month):
            return self._find_k_guess(*self._subset(month))

        Df_list = list()
        F_list = list()
        with PPool(SIZE_POOL) as p:
            F_list = p.map( _reid_multi, [i for i in range(13)])
        F_list = sorted(F_list, key=lambda x : x['month'], reverse = False)
        dtypes = {'id_user': str}
        user_id = pd.DataFrame(sorted(self._ground_truth["id_user"].unique()))
        user_id.columns = ['id_user']
        user_id = user_id.set_index('id_user')
        Df_list.append(user_id)
        for dict_month in F_list:
            col = dict_month.pop("month")
            df = pd.DataFrame.from_dict(dict_month, orient='index')
            df.columns = [col]
            Df_list.append(df)
        F_file = pd.concat(Df_list, axis=1, join_axes=[Df_list[0].index])
        F_file = F_file.reset_index()
        F_file = F_file.fillna('DEL')

        score = self.compare_f_files(F_file)
        self._current_score.append(score)

        return score

    def _calc_sim_mat_dist(self, item_item_dic1, item_item_dic2):
        """ Calcul the distance between two item_item matrix.

        return: the distance between the matrix, max value is 1
        """
        sim_dist = 0
        item_item_dic1_sum = 0

        for item_no, item2_no in item_item_dic1:
            item_item_dic1_sum += item_item_dic1[(item_no, item2_no)]

            if (item_no, item2_no) in item_item_dic2:
                sim_dist += math.fabs(item_item_dic1[(item_no, item2_no)] - item_item_dic2[(item_no, item2_no)])
            else:
                sim_dist += item_item_dic1[(item_no, item2_no)]

        sim_dist /= item_item_dic1_sum

        if sim_dist > 1.0:
            sim_dist = 1.0
        return sim_dist

    def _compare_date_gt_anon(self):
        """Compare each date in T and AT, it's mean to value the transformation applied to the date
        in AT.

        :returns: The total of all the difference between the date in T and AT

        """
        score = 0

        for idx in range(self._anon_trans.shape[0]):
            # Skip this loop if id_user == DEL
            if self._anon_trans.iloc[idx, self._gt_t_col_it['id_user']-1] == "DEL":
                continue

            # Get the date from the data at index idx
            #  TODO: .iloc should be used here for safety reason but iloc does not keep columns
            #  value <12-06-18, Antoine> #
            gt_day = self._ground_truth.loc[idx, self._gt_t_col['date']]
            anon_day = self._anon_trans.loc[idx, self._gt_t_col['date']]

            #  TODO: Mettre ça dans un fichier qui test tous les formats et fait des messages
            #  d'erreurs approprié  <12-06-18, Antoine> #
            gt_day = pd.datetime.strptime(gt_day, '%Y/%m/%d')
            anon_day = pd.datetime.strptime(anon_day, '%Y/%m/%d')

            score += abs((gt_day - anon_day).days)

        score = np.round(float(score)/float(31 * self._anon_trans.shape[0]), 10)

        return score

    def _compare_price_gt_anon(self):
        """Compare each price in T and AT, it's mean to value the transformation applied to the
        price in AT.

        :returns: The total of all the difference between the price in T and AT

        """
        score = 0

        for idx in range(self._anon_trans.shape[0]):
            # Skip this loop if id_user == DEL
            if self._anon_trans.iloc[idx, self._gt_t_col_it['id_user']-1] == "DEL":
                continue

            # Get the date from the data at index idx
            gt_price = float(self._ground_truth.loc[idx, self._gt_t_col['price']])
            anon_price = float(self._anon_trans.loc[idx, self._gt_t_col['price']])

            # Get the difference between date1 and date2 in days
            score += (1 - min(gt_price, anon_price)/max(gt_price, anon_price))

        score = np.round(float(score)/float(self._anon_trans.shape[0]), 10)

        return score

    def _compute_median_qty(self):
        """Compute the median of all qty of item buyed

        :return: median of all qty of item buyed
        """
        # Creating item x user sparse matrix for the ground_truth
        gt_id_item_id_user = self.ground_truth.groupby(
            [self._gt_t_col['id_item'], self._gt_t_col['id_user']]
        )[self._gt_t_col['qty']].sum()

        # Recovering median
        median = gt_id_item_id_user.median()

        return median

    def _collaborative_filtering_item_user(self, data, e2=False):
        """Compute the matrix of cosine similarity between all items

        :data: dataframe from which to compute the collaborative_filtering
        :returns: matrix |col_item|x|col_item|

        """

        id_items_ori = self.ground_truth[self.gt_t_col["id_item"]].unique()

        # Creating item x user sparse matrix for the ground_truth
        data_id_item_id_user = data.groupby(
            [self._gt_t_col['id_item'], self._gt_t_col['id_user']]
        )[self._gt_t_col['qty']].sum()

        if e2:
            # item x user < median
            data_id_item_id_user = data_id_item_id_user[
                data_id_item_id_user < self._compute_median_qty()
            ]

        # Create dataframe matrix
        data_id_item_id_user = data_id_item_id_user.unstack(level=1).to_sparse()

        items_differ = set(id_items_ori) - set(data_id_item_id_user.index)

        if items_differ:
            df_temp = pd.DataFrame(index=items_differ, columns=data_id_item_id_user.columns)
            data_id_item_id_user = data_id_item_id_user.append(df_temp)

        data_id_item_id_user = data_id_item_id_user.sort_index()

        return cosine_similarity(data_id_item_id_user.fillna(0))

    def _e1_metric(self):
        """ Construct a similarity matrix of item buyed (User that have bought this item also bought
        item_i). Here the score is maximized if the quantity is high (calculated by dozen). We
        calculate the difference between the two matrix of item buyed as a score.

        More precisly we construct two matrix M1 and M2, one for the original dataset and one for
        the anonymised one. Both are of size `n x n` where `n` is the number of item. For M_ij
        represent the number of people who have bought the item i and have also bought the item j.

        This procede is called a collaborative filtering
        (https://en.wikipedia.org/wiki/Collaborative_filtering).

        :returns: score of the metric.

        """

        # Copy dataframe
        ground_truth = self._ground_truth.copy()
        anon_trans = self._anon_trans.copy()
        anon_trans = anon_trans.drop(
            anon_trans[anon_trans[self._gt_t_col['id_user']] == 'DEL'].index, axis=0
        ).reset_index(drop=True)

        #  TODO: can be done one time only  <21-12-18, yourname> #
        anon_trans[self._gt_t_col['qty']] = anon_trans[self._gt_t_col['qty']].apply(int)

        # Compute the cosinus similarity item x item
        gt_cos_sim = self._collaborative_filtering_item_user(ground_truth)

        # Compute the cosinus similarity item x item
        at_cos_sim = self._collaborative_filtering_item_user(anon_trans)

        # Compute score
        diff_cos = abs(gt_cos_sim - at_cos_sim)
        score = min(1, diff_cos.sum() / gt_cos_sim.sum())

        # Add the score to the global score for this metric
        self._current_score.append(score)

        return score

    def _e2_metric(self):
        """ Construct a similarity matrix of item buyed (User that have bought this item also bought
        item_i). Here the score is maximized if the quantity is low (<= threshold). We
        calculate the difference between the two matrix of item buyed as a score.

        More precisly we construct two matrix M1 and M2, one for the original dataset and one for
        the anonymised one. Both are of size `n x n` where `n` is the number of item. For M_ij
        represent the number of people who have bought the item i and have also bought the item j.

        This procede is called a collaborative filtering
        (https://en.wikipedia.org/wiki/Collaborative_filtering).

        :returns: score of the metric.

        """

        if True:
            return 0

        # Copy dataframe
        ground_truth = self._ground_truth.copy()
        anon_trans = self._anon_trans.copy()
        anon_trans = anon_trans.drop(
            anon_trans[anon_trans[self._gt_t_col['id_user']] == 'DEL'].index, axis=0
        ).reset_index(drop=True)

        #  TODO: can be done one time only  <21-12-18, yourname> #
        anon_trans[self._gt_t_col['qty']] = anon_trans[self._gt_t_col['qty']].apply(int)

        # Compute the cosinus similarity item x item
        gt_cos_sim = self._collaborative_filtering_item_user(ground_truth, e2=True)

        # Compute the cosinus similarity item x item
        at_cos_sim = self._collaborative_filtering_item_user(anon_trans, e2=True)

        # Compute score
        diff_cos = abs(gt_cos_sim - at_cos_sim)
        score = min(1, diff_cos.sum() / gt_cos_sim.sum())

        # Add the score to the global score for this metric
        self._current_score.append(score)

        return score

    def _e3_metric(self):
        """ Caluclate the difference (as in set difference) and similarity matrix between top-`k`
        items bought from ground truth and anonymised dataset.

        :returns: score of the metric.

        """

        # Copy dataframe
        ground_truth = self._ground_truth.copy()
        anon_trans = self._anon_trans.copy()
        anon_trans = anon_trans.drop(
            anon_trans[anon_trans[self._gt_t_col['id_user']] == 'DEL'].index, axis=0
        ).reset_index(drop=True)

        #  TODO: can be done one time only  <21-12-18, yourname> #
        anon_trans[self._gt_t_col['qty']] = anon_trans[self._gt_t_col['qty']].apply(int)

        # Computing top 5% of most purchased item by customer
        item_count = ground_truth.groupby(self._gt_t_col['id_item']).size()
        gt_top_k = list(
            item_count.sort_values(ascending=False).head(int(item_count.shape[0]*0.05)).index
        )

        # Ground Truth is now with top k items
        ground_truth = ground_truth.set_index(self._gt_t_col['id_item'])
        ground_truth = ground_truth.loc[gt_top_k]
        ground_truth = ground_truth.reset_index()

        # Compute the cosinus similarity item x item
        gt_cos_sim = self._collaborative_filtering_item_user(ground_truth)

        # Computing top 5% of most purchased item by customer
        item_count = anon_trans.groupby(self._gt_t_col['id_item']).size()
        at_top_k = list(
            item_count.sort_values(ascending=False).head(int(item_count.shape[0]*0.05)).index
        )

        # Anonymized File is now with top k items
        anon_trans = anon_trans.set_index(self._gt_t_col['id_item'])
        anon_trans = anon_trans.loc[at_top_k]
        anon_trans = anon_trans.reset_index()

        # Compute the cosinus similarity item x item
        at_cos_sim = self._collaborative_filtering_item_user(anon_trans)

        # Compute score
        diff_cos = min(1, (abs(gt_cos_sim - at_cos_sim).sum()) / gt_cos_sim.sum())
        # Jaccard distance
        diff_top_k = (
            len(set(gt_top_k).union(at_top_k)) - len(set(gt_top_k).intersection(set(at_top_k)))
        )/ len(set(gt_top_k).union(at_top_k))

        score = max(diff_cos, diff_top_k)

        # Add the score to the global score for this metric
        self._current_score.append(score)

        return score

    def _e4_metric(self):
        """ Calculate the mean distance in day between anonymised and ground truth
        transactions.

        :returns: score of the metric.

        """
        score = self._compare_date_gt_anon()
        self._current_score.append(score)

        return score

    def _e5_metric(self):
        """ Calculate the difference, as the ratio, of all item prices.

        :returns: score of the metric.

        """
        score = self._compare_price_gt_anon()
        self._current_score.append(score)

        return score

    def _e6_metric(self):
        """ Calculate the ratio between the number of lines removed in the anonymized table over the
        number of lines in the original dataset.

        :returns: score of the metric.

        """
        score = np.round(float(1 - self._anonymized.shape[0]/self._ground_truth.shape[0]), 4)
        self._current_score.append(score)

        return score

    @property
    def f(self):
        """
        Get the original file F.
        """
        return self._f_orig


    @property
    def users(self):
        """
        Get the users data
        """
        return self._users

    @property
    def ground_truth(self):
        """
        Get the ground_truth data
        """
        return self._ground_truth

    @property
    def anonymized(self):
        """
        Get the anonymized data
        """
        return self._anonymized

    @property
    def users_t_col(self):
        """
        Get the column used in the M csv
        """
        return self._users_t_col

    @property
    def gt_t_col(self):
        """
        Get the column used in the T csv
        """
        return self._gt_t_col

    @property
    def current_score(self):
        """
        Get the current score calculated by the metrics
        """
        return self._current_score

def metric_wrapper(metric, instance, numero):
    """Launch a metric in function of instance metric and the number of the later.

    :metric: a single char wich is 's' or 'e', respectivly for reid and utility metrics.
    :instance: the instance of a Metric class containing methods `metric`.
    :numero: the ieme method of the instance you want to call.

    :returns: Result of the metric method called.

    """
    method = "_{}{}_metric".format(metric, numero)
    return getattr(instance, method)()

def utility_metric(ground_truth, sub):
    """TODO: Docstring for utility_metric.

    :arg1: TODO
    :returns: TODO

    """
    # Initialize utility metrics
    metric = Metrics(ground_truth, sub)

    #Compute utility metrics as subprocesses
    metric_pool = Pool()
    utility_wrapper = partial(metric_wrapper, "e", metric)
    utility_scores = metric_pool.map(utility_wrapper, range(1, 7))

    #Compute reidentification metrics as subprocesses
    #metric_pool = Pool()
    #reid_wrapper = partial(metric_wrapper, "s", metric)
    #reid_scores = metric_pool.map(reid_wrapper, range(1, 8))

    return utility_scores
